In [1]:
from keras.preprocessing.image import ImageDataGenerator
import os

def image_generator(image_class):
    datagen = ImageDataGenerator(zoom_range = 0.2,
                            height_shift_range = 0.25,
                            width_shift_range = 0.25,
                            rotation_range = 25)

    i = 0
    for batch in datagen.flow_from_directory('dataset/trainingSet/', target_size = (32, 32),
                                                     batch_size = 32,
                                                     classes=[image_class],
                                                     class_mode="binary",
                                                     shuffle=True,
                                                     save_to_dir='dataset/augmented/'+image_class+'/', save_prefix='augmented'):
        i += 1
        if i > 60: # save 60 images for each batch
            break  # otherwise the generator would loop indefinitely

for i in range(10):
    os.mkdir('dataset/augmented/'+str(i))
    image_generator(str(i))

Using TensorFlow backend.


Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.


In [9]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from numpy import zeros, array, newaxis, swapaxes
import numpy as np
from PIL import Image
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = False
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(_, _), (x_test, y_test) = cifar10.load_data()


x_train=zeros([0,3,32,32])
y_train=zeros([0,1])

for x in range(10):
    for filename in os.listdir('dataset/trainingSet/'+str(x)+'/'):
        img = Image.open('dataset/trainingSet/'+str(x)+'/'+filename)
        arr = array(img)
        arr = swapaxes(arr,1,2)
        arr = swapaxes(arr,0,1)
        arr = arr[newaxis, :]
        arr2 = array([x])
        arr2 = arr2[newaxis, :]
        
        x_train = np.append(x_train, arr, axis=0)
        y_train = np.append(y_train, arr2, axis=0)
        
        
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (5000, 3, 32, 32)
5000 train samples
10000 test samples


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_5/MaxPool' (op: 'MaxPool') with input shapes: [?,1,30,32].

In [13]:
from keras.preprocessing import image

for filename in os.listdir('dataset/augmented/0'):
    test_image = image.load_img('dataset/augmented/0/'+filename, target_size = (32, 32))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    print(result[0][class_name]==1)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.8440728e-30 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 4.2222767e-11]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0.000000e+00 6.713025e-36 0.000000e+00 1.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[5.0749708e-07 0.0000000e+00 4.2245155e-33 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 9.9999952e-01 0.0000000e+00]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 9.861405e-28 0.000000e+00]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 4.1620863e-25]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[9.9997926e-01 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 2.0711406e-05]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 2.975208e-13 0.000000e+00]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1.2744611e-37 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+0

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.5491321e-22 0.0000000e+00]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 5.408362e-15 0.000000e+00]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.5918917e-24 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
[7.5080153e-17 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[9.999999e-01 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 6.873261e-08 0.000000e+00]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[4.6534686e-15 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.9934975  0.         0.         0.         0.         0.
 0.         0.         0.00650245 0.        ]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.7472483e-27 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1.1047992e-21 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.000000

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[3.4578767e-21 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00
 2.5757395e-37 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0.00000

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 5.449396e-12 0.000000e+00 1.000000e+00]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[5.29393e-08 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 1.00000e+00 0.00000e+00]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[1.1514517e-19 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00 0.000000

In [37]:
def clear_augmented_images(class_name):
    for filename in os.listdir('dataset/augmented/'+str(class_name)):   
        test_image = image.load_img('dataset/augmented/'+str(class_name)+'/'+filename, target_size = (32, 32))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model.predict(test_image)
        
        
        print('Class:' + str(class_name) + '  Prediction:' + str(result[0]))
        
        if result[0][class_name] != 1:
            os.remove('dataset/augmented/'+str(class_name)+'/'+filename)
            print('Removed:' + filename)
            

for i in range(10):
    clear_augmented_images(i)


Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_0_2882441.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1.8440728e-30 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 4.2222767e-11]
Removed:augmented_100_289104.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
Removed:augmented_100_9042333.png
Class:0  Prediction:[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_101_2138450.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_101_4811795.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_101_8725251.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Pre

Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_145_2018648.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_145_3840791.png
Class:0  Prediction:[1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 2.2902356e-31 0.0000000e+00]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_146_3854910.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_148_3358227.png
Class:0  Prediction:[1. 0

Class:0  Prediction:[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_18_4372426.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Removed:augmented_18_8756390.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_190_8613187.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_190_882054.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Removed:augmented_191_3608706.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Removed:augmented_191_5846963.png
Class:0  Prediction:[1.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 4.5322304e-32 0.0000000e+00]
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Removed:augmented_191_8577968.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 

Removed:augmented_232_5931083.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_233_4419396.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_233_5088397.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[6.485729e-11 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00]
Removed:augmented_234_7213737.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_234_872974.png
Class:0  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_235_1659019.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0.000000e+00 3.836542e-20 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00]
Removed:augmented_235_5

Removed:augmented_277_5143751.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_278_1197640.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_278_248936.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_279_2106055.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_27_2379471.png
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_27_2755803.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Class:0  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_27_8795175.png
Class:0  Prediction:[1. 0. 0. 0. 0. 0.

...

Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_403_5935567.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_403_8983321.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_403_9418117.png
Class:9  Prediction:[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Removed:augmented_404_1400484.png
Class:9  Prediction:[2.4722187e-17 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00]
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_404_6066281.png
Class:9  Prediction:[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 6.893878e-37 1.000000e+00 0.000000e+00 0.000000e+00]
Removed:augmented_405_316461.png
Class:9  Prediction:[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 2.7375114e-23]
Removed:augmented_405_3465031.png
Class:9  Prediction:[0. 

Class:9  Prediction:[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Removed:augmented_424_9694856.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_425_6392081.png
Class:9  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_425_7392398.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_426_3543784.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Removed:augmented_427_523771.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Removed:augmented_427_7240093.png
Class:9  Prediction:[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_427_8192139.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_428_2482034.png
C

Removed:augmented_465_5116756.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_465_5204661.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0092674e-21 0.0000000e+00 1.0000000e+00]
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_466_7989084.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_466_8502055.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_467_101529.png
Class:9  Prediction:[2.1870574e-05 0.0000000e+00 9.9997818e-01 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
Removed:augmented_467_1329565.png
Class:9  Prediction:[0.0000000e+00 1.0038943e-31 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00]
Class:9  Prediction:[0. 0. 0. 1. 0. 0. 0. 0. 0. 

Removed:augmented_483_9613576.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_484_1861521.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_484_3261403.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Removed:augmented_484_8027637.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_485_4270396.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_485_5217907.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_485_8356577.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_486_1411822.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Removed:augmented_486_6001526.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_486_7225827.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_487_6605670.png
Cl

Removed:augmented_73_3062872.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_73_3106104.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_73_5957353.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_76_8993865.png
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Removed:augmented_76_899903.png
Class:9  Prediction:[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Class:9  Prediction:[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Remo